# Segunda entrega - 51940 Cueva Tomas
Este notebook contiene

In [1]:
!pip install numpy pandas requests SQLAlchemy psycopg2-binary

In [9]:
from os import environ as env
from pyspark.sql import SparkSession
from etl import(
    extract_CoinGecko_API as extract,
    transform_CoinGecko_API as transform,
    load_pyspark_redshift_connectr as load
)
import sqlalchemy
import pandas as pd
import requests
import numpy as np

In [10]:
extr = extract.Extract("https://api.coingecko.com/api/v3/")
transf = transform.Transform()
ld = load.Load()

/home/coder/working_dir/spark_drivers/postgresql-42.5.2.jar


### 100 criptos con mayor capitalización bursatil

In [11]:
json_top = extr.get_criptos_top()

In [12]:
df_top = transf.transformation_top(json_top)

In [13]:
df_top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                100 non-null    object 
 1   symbol                            100 non-null    object 
 2   name                              100 non-null    object 
 3   current_price                     100 non-null    float64
 4   market_cap                        100 non-null    int64  
 5   market_cap_rank                   100 non-null    int64  
 6   total_volume                      100 non-null    float64
 7   high_24h                          100 non-null    float64
 8   low_24h                           100 non-null    float64
 9   price_change_24h                  100 non-null    float64
 10  price_change_percentage_24h       100 non-null    float64
 11  market_cap_change_24h             100 non-null    float64
 12  market_ca

In [14]:
table = "criptos_market_cap"

In [15]:
ld.execute(df_top, table)

Ejecutando ETL
Convertir el DataFrame de pandas a un PySpark DataFrame
DataFrame[id: string, symbol: string, name: string, current_price: double, market_cap: bigint, market_cap_rank: bigint, total_volume: double, high_24h: double, low_24h: double, price_change_24h: double, price_change_percentage_24h: double, market_cap_change_24h: double, market_cap_change_percentage_24h: double, circulating_supply: double, ath: double, ath_change_percentage: double, ath_date: string, atl: double, atl_change_percentage: double, atl_date: string, last_updated: string]
Cargar el PySpark DataFrame en Redshift
Se produjo excepción An error occurred while calling o98.save.
: java.lang.ClassNotFoundException: com.amazon.redshift.jdbc.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:587)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:520)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegi